In [1]:
마포구 = [1144010100,1144010200,1144010300,1144010400,1144010500,1144010600,1144010700,1144010800,1144010900,1144011000,1144011100,1144011200,1144011300,1144011400,1144011500,1144011600,1144011700,1144011800,1144012000,1144012100,1144012200,1144012300,1144012400,1144012500,1144012600,1144012700]
mapo = ['아현동','공덕동','신공덕동','도화동','용강동','토정동','마포동','대흥동','염리동','노고산동','신수동','현석동','구수동','창전동','상수동','하중동','신정동','당인동','서교동','동교동','합정동','망원동','연남동','성산동','중동','상암동']

In [2]:
import requests #필요한 모듈들을 import 합니다.
from bs4 import BeautifulSoup
import pandas as pd

names = []
areas = []
prices = []
area_per_price = []
for area_code in 마포구:
    
    url = 'https://hogangnono.com/region/{}'.format(area_code) #url을 area_code를 이용하여 반복문으로 한번에 모두 가져옵니다.
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    name = []
    for pages in soup.findAll('li'):    # 아파트 이름을 가져옵니다
        for link in pages('h5',{'class':'name'}):
            data = link.text        
            name.append(data)
    area = []    
    for pages in soup.findAll('li'):    # 아프트 면적을 가져옵니다
        for link in pages('span',{'class':'area'}):
            data = link.text        
            area.append(data)
    price = []
    for pages in soup.findAll('li'):    #아파트 가겨을 가져옵니다
        for link in pages('span',{'class':'price'}):
            data = link.text        
            price.append(data)
            
    index_dash = [i for i, n in enumerate(price) if n == '-'] # 아파트 가격이 '-' 로 나와있는 아파트들의 index를 찾아냅니다
    for i in index_dash[::-1]: # '-'로 나와있는 아파트를 리스트에서 지웁니다
        del(name[i])
        del(area[i])
        del(price[i])
    
    new_name = [] # 아파트 이름 앞에 동 이름을 제거합니다
    for k in name:
        k_idx = k.find(' ')
        k = k[k_idx+1:]
        new_name.append(k)
        
    
    int_area = []
    for l in area: # 아파트 면적을 integer로 바꿔줍니다
        per = l[:-1]
        int_area.append(int(per))
    
    new_price = []
    for j in price: # 아파트 가격을 integer 로 바꿔줍니다

        if '억'in j:
            if len(j)>4:
                y_idx = j.find('억')
                pri = j[0:y_idx]
                pri1 = j[y_idx+1:]
                pri2 = pri1.replace(',','')
                j = ((int(pri)*10000)+(int(pri2)))*10000
                new_price.append(j)
            else:
                j = j.replace('억','')
                j = int(j)*100000000
                new_price.append(j)

        else:
            j = j.replace(',','')
            j = int(j)*10000
            new_price.append(j)
            
    per_price = [] # 평당 가격을 계산합니다
    for a in range(len(new_price)):
        perp = new_price[a]/int_area[a]
        per_price.append(round(perp))
        
    
    
    names.append(new_name) # lists in lists 로 저장합니다
    areas.append(int_area)
    prices.append(new_price)
    area_per_price.append(per_price)
    
amean = []

for i in range(len(mapo)): # 동별로 평당가격의 mean을 구해줍니다
    if len(area_per_price[i])>0:
        mean = sum(area_per_price[i])/len(area_per_price[i])
        amean.append(round(mean))
        
    else:
        amean.append(0)

kk = {'dong':마포구,'per price':amean}
df = pd.DataFrame(kk,index = mapo)
df

,dong,per price
아현동,1144010100,26427202
공덕동,1144010200,21764240
신공덕동,1144010300,25635697
도화동,1144010400,20690426
용강동,1144010500,37924038
토정동,1144010600,32250000
마포동,1144010700,19504804
대흥동,1144010800,28628065
염리동,1144010900,31728089
노고산동,1144011000,14521535


In [3]:
df.to_csv('mapogu.csv')